# Xception 15 clases

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


Xception for 
apple_pie/       beef_tartare/  bread_pudding/      cannoli/
baby_back_ribs/  beet_salad/    breakfast_burrito/  caprese_salad/
baklava/         beignets/      bruschetta/
beef_carpaccio/  bibimbap/      caesar_salad/

In [0]:
ls prueba

apple_pie/       beef_tartare/  bread_pudding/      cannoli/
baby_back_ribs/  beet_salad/    breakfast_burrito/  caprese_salad/
baklava/         beignets/      bruschetta/
beef_carpaccio/  bibimbap/      caesar_salad/


In [0]:
train_data_dir='prueba'

In [0]:
TEST_SPLIT = 0.25
epochs = 80
#batch_size = 16
batch_size=32
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (112, 112)
LEARNING_RATE = 0.001

In [0]:

train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 11251 images belonging to 15 classes.
Found 2811 images belonging to 15 classes.


In [0]:
validation_generator

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 11251 images belonging to 15 classes.
Found 2811 images belonging to 15 classes.


In [0]:
from keras import applications
model = applications.Xception(weights='imagenet', 
                                include_top=False, input_shape = (112,112,3))

83689472/83683744 [==============================] - 10s 0us/step


In [0]:
len(model.layers)

132

In [0]:
for layer in model.layers[:50]:
    layer.trainable = False
regparameter=0.0001

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.4)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(15, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 55, 55, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 55, 55, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 55, 55, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

/home/davidrendon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [0]:
chkpt = ModelCheckpoint(filepath='best_model_todatefreeze50.hdf5', save_best_only=True, save_weights_only=True)

In [0]:
tf.keras.callbacks.ModelCheckpoint

tensorflow.python.keras.callbacks.ModelCheckpoint

In [0]:
pwd

'/home/davidrendon/Documents/proyectou3deep'

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 100,callbacks=[chkpt])

Epoch 1/100
351/351 [==============================] - 288s 821ms/step - loss: 0.4430 - accuracy: 0.9735 - val_loss: 2.4990 - val_accuracy: 0.5420
Epoch 2/100
351/351 [==============================] - 286s 815ms/step - loss: 0.1903 - accuracy: 0.9937 - val_loss: 4.2259 - val_accuracy: 0.5847
Epoch 3/100
351/351 [==============================] - 286s 814ms/step - loss: 0.1714 - accuracy: 0.9935 - val_loss: 3.2047 - val_accuracy: 0.5682
Epoch 4/100
351/351 [==============================] - 286s 814ms/step - loss: 0.1793 - accuracy: 0.9934 - val_loss: 5.2152 - val_accuracy: 0.5175
Epoch 5/100
351/351 [==============================] - 286s 815ms/step - loss: 0.1830 - accuracy: 0.9930 - val_loss: 3.1725 - val_accuracy: 0.5772
Epoch 6/100
351/351 [==============================] - 285s 812ms/step - loss: 0.1700 - accuracy: 0.9941 - val_loss: 3.4862 - val_accuracy: 0.5617
Epoch 7/100
351/351 [==============================] - 285s 813ms/step - loss: 0.1559 - accuracy: 0.9955 - val_loss: 3

351/351 [==============================] - 285s 812ms/step - loss: 0.1499 - accuracy: 0.9934 - val_loss: 1.5106 - val_accuracy: 0.5747
Epoch 57/100
351/351 [==============================] - 286s 815ms/step - loss: 0.1593 - accuracy: 0.9957 - val_loss: 5.9862 - val_accuracy: 0.5804
Epoch 58/100
351/351 [==============================] - 286s 813ms/step - loss: 0.1542 - accuracy: 0.9963 - val_loss: 3.9083 - val_accuracy: 0.5905
Epoch 59/100
351/351 [==============================] - 285s 813ms/step - loss: 0.1387 - accuracy: 0.9972 - val_loss: 3.8139 - val_accuracy: 0.5747
Epoch 60/100
351/351 [==============================] - 286s 814ms/step - loss: 0.1496 - accuracy: 0.9947 - val_loss: 3.7481 - val_accuracy: 0.5934
Epoch 61/100
351/351 [==============================] - 285s 812ms/step - loss: 0.1351 - accuracy: 0.9963 - val_loss: 1.9581 - val_accuracy: 0.5729
Epoch 62/100
351/351 [==============================] - 286s 814ms/step - loss: 0.1452 - accuracy: 0.9955 - val_loss: 3.6046 

In [0]:
model_final.save('xception15classacc59'
                )

In [0]:
model_final.predict_generator(generator=validation_generator)



import numpy as np
ypred=np.argmax(prob,axis=1)

y_true=validation_generator.classes

tf.math.confusion_matrix(y_true,ypred)

